<h1 align="center">Machine learning-based prediction of early recurrence in glioblastoma patients: a glance towards precision medicine <br><br>[Random Forest]</h1>

<h2>[1] Library</h2>

In [ ]:
# OS library
import os
import sys
import argparse
import itertools
import random

# Analysis
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Sklearn
from boruta import BorutaPy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score, recall_score, classification_report, accuracy_score, auc, roc_curve
from sklearn.model_selection import RandomizedSearchCV

import scikitplot as skplt
from imblearn.over_sampling import RandomOverSampler, SMOTENC, SMOTE

<h2>[2] Exploratory data analysis and Data Preprocessing</h2>

<h4>[-] Load the database</h4>

In [ ]:
file = os.path.join(sys.path[0], "db.xlsx")
db = pd.read_excel(file)

print("N° of patients: {}".format(len(db)))
print("N° of columns: {}".format(db.shape[1]))
db.head()

<h4>[-] Drop unwanted columns + create <i>'results'</i> column</h4>

In [ ]:
df = db.drop(['Name_Surname','...'], axis = 'columns')

print("Effective features to consider: {} ".format(len(df.columns)-1))
print("Creating 'result' column...")

# 0 = No relapse
df.loc[df['PFS'] > 6, 'result'] = 0

# 1 = Early relapse (within 6 months)
df.loc[df['PFS'] <= 6, 'result'] = 1

df.head()

<h4>[-] Check for class imbalance in the <i>'results'</i> column </h4>

In [ ]:
print("PFS Overview")
print(df.result.value_counts())

df.result.value_counts().plot(kind='pie', autopct='%1.0f%%', colors=['skyblue', 'orange'], explode=(0.05, 0.05))

<h4>[-] Label encoding of the categorical variables </h4>

In [ ]:
df['sex'] =df['sex'].astype('category')
df['ceus'] =df['ceus'].astype('category')
df['ala'] =df['ala'].astype('category')

#df['Ki67'] =df['Ki67'].astype(int)
df['MGMT'] =df['MGMT'].astype('category')
df['IDH1'] =df['IDH1'].astype('category')

df['side'] =df['side'].astype('category')
df['ependima'] =df['ependima'].astype('category')
df['cc'] =df['cc'].astype('category')
df['necrotico_cistico'] =df['necrotico_cistico'].astype('category')
df['shift'] =df['shift'].astype('category')

## VARIABLE TO ONE-HOT-ENCODE
df['localization'] =df['localization'].astype(int)
df['clinica_esordio'] =df['clinica_esordio'].astype(int)
df['immediate_p_o'] =df['immediate_p_o'].astype(int)
df['onco_Protocol'] =df['onco_Protocol'].astype(int)

df['result'] =df['result'].astype(int)

dummy_v = ['localization', 'clinica_esordio', 'onco_Protocol', 'immediate_p_o']

df = pd.get_dummies(df, columns = dummy_v, prefix = dummy_v)

<h4>[-] Evaluate variables' correlation with <u>'PFS'</u> columns </h4>

In [ ]:
corr = df.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=60,
    horizontalalignment='right'
);

<h2>[2] Prediction Models</h2>

<h4> [-] Training and testing set splitting</h4>

In [5]:
target = df['result']
inputs = df.drop(['result', 'PFS'], axis = 'columns')
x_train, x_test, y_train, y_test = train_test_split(inputs['...'],target,test_size=0.20, random_state=10)

NameError: name 'df' is not defined

<h4> [-] BORUTA Features Selection</h4>

In [ ]:
x = x_train.values
y = y_train.values
y = y.ravel()

rf_boruta = RandomForestClassifier(n_jobs=-1, class_weight={0:1, 1:3}, max_depth=3)
feat_selector = BorutaPy(rf_boruta, n_estimators='auto', verbose=0, random_state=42, perc='...')
feat_selector.fit(x, y)

cols = inputs.columns[feat_selector.support_]
print("N° of selected features: {}".format(len(cols)))
print(cols) 

<h4> [-] Random Grid Search Hyperparameter tuning</h4>

In [ ]:
# The function to measure the quality of a split
criterion = ['gini', 'entropy']

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 50, num = 5)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(14, 30, num = 2)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [ 2, 3, 4, 5, 8]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 3, 4, 5, 6]

max_leaf_nodes = [None, 2, 3, 4, 5, 6]
# Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid = {'criterion': criterion,
               'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'max_leaf_nodes': max_leaf_nodes,
               'bootstrap':bootstrap
              }

# First create the base model to tune
rf = RandomForestClassifier(random_state=42,
                            n_jobs = -1, 
                            class_weight=class_weight)

# Random search of parameters, using 3 fold cross validation, different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 500, 
                               cv = 5)
# Fit the random search model
rf_random.fit(x_train, y_train)
rf_random.best_params_

<h4>[-] SMOTE-NC</h4>

In [ ]:
smote_nc = SMOTENC(categorical_features=[3,4,10,11], k_neighbors= 3, random_state=42)
x_smote_train, y_smote_train = smote_nc.fit_resample(x_train, y_train)

<h4>[-] Random Forest Model</h4>

In [ ]:
rm_smote = RandomForestClassifier(random_state = 42,
                                       criterion= '...',
                                       n_estimators = '...',
                                       min_samples_split = '...',
                                       min_samples_leaf = '...',
                                       max_leaf_nodes = '...',
                                       max_features = '...',
                                       max_depth = '...',
                                       bootstrap = '...')

rm_smote.fit(x_smote_train, y_smote_train)
print("Trained \n")

score_rf_smote = rm_smote.score(x_test, y_test)
print("Random Forest accuracy: ", round(score_rf_smote*100,2), "% \n")

y_smote_predicted = rm_smote.predict(x_test)
cm_rf_smote = confusion_matrix(y_test, y_smote_predicted)
print(cm_rf_smote, "\n")

false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_smote_predicted)
roc_auc = auc(false_positive_rate, true_positive_rate)

print('1. The F-1 Score of the model {} \n '.format(round(f1_score(y_test, y_smote_predicted, average = 'macro'), 2)))
print('2. The Recall Score of the model {} \n '.format(round(recall_score(y_test, y_smote_predicted, average = 'macro'), 2)))
print('3. Classification report \n {} \n'.format(classification_report(y_test, y_smote_predicted)))
print('3. AUC: \n {} \n'.format(roc_auc))

tn, fp, fn, tp = cm_rf_smote.ravel()

# Sensitivity, hit rate, Recall, or true positive rate
tpr = tp/(tp+fn)
print("Sensitivity (TPR): {}".format(tpr))

# Specificity or true negative rate
tnr = tn/(tn+fp)
print("Specificity (TNR): {}".format(tnr))

# Precision or positive predictive value
ppv = tp/(tp+fp)
print("Precision (PPV): {}".format(ppv))

# Negative predictive value
npv = tn/(tn+fn)
print("Negative Predictive Value (NPV): {}".format(npv))

# False positive rate
fpr = fp / (fp + tn)
print("False Positive Rate (FPV): {}".format(fpr))

<h4> [-] Features Importance Plot </h4>

In [ ]:
features = x_train.columns.values

features[0] = 'Age'
features[6] = 'Tumor volume T1'
features[7] = 'Tumor and edema volume T2'
features[8] = 'Residual tumor'
features[9] = 'Pre-operative KPS'
features[10] = 'Post-operative KPS'
features[11] = 'Onset neurological symptoms = 1'
features[12] = 'Oncological protocol = 0'
features[13] = 'Oncological protocol = 1'
features[14] = 'Oncological protocol = 2'

indices = np.argsort(importances)

plt.title('Random Forest Classifier Features Importance')
plt.barh(range(len(indices)), importances[indices], color='g', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')

plt.savefig("RF Features importance.jpg", dpi = 400, facecolor='w', edgecolor='w',
        orientation='landscape', papertype=None, format=None,
        transparent=False, bbox_inches='tight', pad_inches=0.3,
        frameon=None)

plt.show()